1- Importing librarie for data manipulation


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

2- Load Data

In [2]:
df = pd.read_csv('data/heart.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


In [64]:
#Check for missing values
df.isnull().sum()   

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [65]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,1025.000000,1025.000000,1025.000000,1025.000000,1025.00000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000
mean,54.434146,0.695610,0.942439,131.611707,246.00000,0.149268,0.529756,149.114146,0.336585,1.071512,1.385366,0.754146,2.323902,0.513171
std,9.072290,0.460373,1.029641,17.516718,51.59251,0.356527,0.527878,23.005724,0.472772,1.175053,0.617755,1.030798,0.620660,0.500070
min,29.000000,0.000000,0.000000,94.000000,126.00000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,48.000000,0.000000,0.000000,120.000000,211.00000,0.000000,0.000000,132.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,56.000000,1.000000,1.000000,130.000000,240.00000,0.000000,1.000000,152.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,275.00000,0.000000,1.000000,166.000000,1.000000,1.800000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.00000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


Attribute Information from https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset/data:
    
    - age

    - sex (0 for female and 1 for Male)

    - chest pain type (4 values)                          --->CATEGORICAL FEATURE

    - resting blood pressure

    - serum cholestoral in mg/dl

    - fasting blood sugar > 120 mg/dl IT TAKES TWO VALUES --->CATEGORICAL FEATURE

    - resting electrocardiographic results (values 0,1,2) --->CATEGORICAL FEATURE

    - maximum heart rate achieved

    - exercise induced angina                             --->CATEGORICAL FEATURE

    - oldpeak = ST depression induced by exercise relative to rest

    - the slope of the peak exercise ST segment           --->CATEGORICAL FEATURE

    - number of major vessels (0-3) colored by flourosopy --->CATEGORICAL FEATURE
    
    - thal: 0 = normal; 1 = fixed defect; 2 = reversable defect --->CATEGORICAL FEATURE

3- Importing scikit learn for the features extraction and model training


In [66]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

In [ ]:
%pip install tqdm
from tqdm.auto import tqdm

Split the dataset

In [68]:
# Split the dataset
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

# Reset indexes
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

# Separate target variable
y_train = df_train.target.values
y_val = df_val.target.values
y_test = df_test.target.values

# Remove target from dataframes
del df_train['target']
del df_val['target']
del df_test['target']

In [69]:
numerical = ['age', 'sex', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical = ['cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

4- Training

In [70]:
def train (df_train, y_train, C=1.0):
    train_dicts = df_train[categorical + numerical].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(train_dicts)
    
    model = LogisticRegression(C=C, max_iter=len(X_train)*10)
    model.fit(X_train, y_train)
    
    return dv, model


In [71]:
dv, model = train(df_train, y_train, C=1.0)

In [72]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')
    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]
    return y_pred

In [73]:
y_pred = predict(df_val, dv, model)

auc = roc_auc_score(y_val, y_pred)
print(f"AUC on validation set: {auc:.3f}")

AUC on validation set: 0.914


5- Tuning differente parameters for training

Manual tuning

In [74]:
n_splits = 5

for C in tqdm([0.001, 0.01, 0.1, 1, 10, 100]):
    aucs = []
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)
    for train_index, val_index in kf.split(df_full_train):
        df_train = df_full_train.iloc[train_index].copy()
        df_val = df_full_train.iloc[val_index].copy()
        
        y_train = df_train.target.values
        y_val = df_val.target.values
        
        df_train = df_train.drop('target', axis=1)
        df_val = df_val.drop('target', axis=1)
        
        dv, model = train(df_train, y_train, C=C)
        y_pred = predict(df_val, dv, model)
        auc = roc_auc_score(y_val, y_pred)
        aucs.append(auc)
    print(f"C={C}: AUC={np.mean(aucs):.3f} +/- {np.std(aucs):.3f}")

 17%|█▋        | 1/6 [00:00<00:01,  3.74it/s]

C=0.001: AUC=0.799 +/- 0.023


 33%|███▎      | 2/6 [00:00<00:01,  2.09it/s]

C=0.01: AUC=0.888 +/- 0.024


 50%|█████     | 3/6 [00:01<00:01,  1.72it/s]

...

C=1: AUC=0.917 +/- 0.027


 83%|████████▎ | 5/6 [00:03<00:00,  1.29it/s]

C=10: AUC=0.917 +/- 0.026


100%|██████████| 6/6 [00:04<00:00,  1.41it/s]

C=100: AUC=0.917 +/- 0.026


Using GridSearchCV, a scikit-learn function that performs hyperparameter tuning automaticaly by training and evaluating the ML model using different combinations of hyperparameters.

In [75]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(
    LogisticRegression(max_iter=len(df_full_train)*10),
    param_grid,
    cv=5,
    scoring='roc_auc',  # or 'accuracy', 'f1', 'roc_auc' etc.
    verbose=1  #show process
    #return_train_score=True #detect overfitting but optional
)
dicts = df_full_train[categorical+numerical].to_dict(orient='records')
dv2 = DictVectorizer(sparse=False)
X_train = dv2.fit_transform(dicts)

y_train = df_full_train.target.values
grid_search.fit(X_train, y_train)
best_C = grid_search.best_params_['C']
best_auc = grid_search.best_score_
print(f"Best AUC: {best_auc:.3f}")
print(f"Best C: {best_C}")

Fitting 5 folds for each of 7 candidates, totalling 35 fits


Best AUC: 0.915
Best C: 1


Choosing the best C: 1 and examine the Evaluation Metrics on the test set

In [76]:
dv, model = train(df_full_train, df_full_train.target.values, C=1)
y_pred = predict(df_test, dv, model)
auc = roc_auc_score(y_test, y_pred)
print(f"AUC on test set: {auc:.3f}\n")

actual_positive = (y_test == 1)
actual_negative = (y_test == 0)

t=0.5
predicted_positive = (y_pred >= t)
predicted_negative = (y_pred < t)

tp = (predicted_positive & actual_positive).sum()
tn = (predicted_negative & actual_negative).sum()
fp = (predicted_positive & actual_negative).sum()
fn = (predicted_negative & actual_positive).sum()

#print(tp)

total_pos = actual_positive.sum()
total_neg = (~actual_positive).sum()

print(f"\nConfusion Matrix on the test dataset c(threshold={t}):")
print(f"{'':>15} Predicted Pos  Predicted Neg")
print(f"Actual Pos      {tp} ({tp/total_pos:.2%})    {fn} ({fn/total_pos:.2%})")
print(f"Actual Neg      {fp} ({fp/total_neg:.2%})    {tn} ({tn/total_neg:.2%})")
print(f"\nMetrics:")
print(f"  Sensitivity/Recall: {tp/total_pos:.3f}")
print(f"  Specificity:        {tn/total_neg:.3f}")
print(f"  Precision:          {tp/(tp+fp):.3f}")
print(f"  Accuracy:           {(tp+tn)/(tp+tn+fp+fn):.3f}")

AUC on test set: 0.921


Confusion Matrix on the test dataset c(threshold=0.5):
                Predicted Pos  Predicted Neg
Actual Pos      86 (89.58%)    10 (10.42%)
Actual Neg      29 (26.61%)    80 (73.39%)

Metrics:
  Sensitivity/Recall: 0.896
  Specificity:        0.734
  Precision:          0.748
  Accuracy:           0.810


6- Try to train the model in another ways

In [77]:
# Display duplicated rows
duplicated_rows = df[df.duplicated()].sort_values(by=list(df.columns))
print(f"Number of duplicated rows: {len(duplicated_rows)}")
#print("\nThe duplicated rows:")
#duplicated_rows                                                                                           

Number of duplicated rows: 723


In [78]:
df = df.drop_duplicates().reset_index(drop=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       302 non-null    int64  
 1   sex       302 non-null    int64  
 2   cp        302 non-null    int64  
 3   trestbps  302 non-null    int64  
 4   chol      302 non-null    int64  
 5   fbs       302 non-null    int64  
 6   restecg   302 non-null    int64  
 7   thalach   302 non-null    int64  
 8   exang     302 non-null    int64  
 9   oldpeak   302 non-null    float64
 10  slope     302 non-null    int64  
 11  ca        302 non-null    int64  
 12  thal      302 non-null    int64  
 13  target    302 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.2 KB


In [79]:
# Split the dataset
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

# Reset indexes
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

# Separate target variable
y_train = df_train.target.values
y_val = df_val.target.values
y_test = df_test.target.values

# Remove target from dataframes
del df_train['target']
del df_val['target']
del df_test['target']

In [80]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(
    LogisticRegression(max_iter=len(df_full_train)*10),
    param_grid,
    cv=5,
    scoring='roc_auc',  # or 'accuracy', 'f1', 'roc_auc' etc.
    verbose=1  #show process
    #return_train_score=True #detect overfitting but optional
)
dicts = df_full_train[categorical+numerical].to_dict(orient='records')
dv3 = DictVectorizer(sparse=False)
X_train = dv3.fit_transform(dicts)

y_train = df_full_train.target.values
grid_search.fit(X_train, y_train)
best_C = grid_search.best_params_['C']
best_auc = grid_search.best_score_
print(f"Best AUC: {best_auc:.3f}")
print(f"Best C: {best_C}")

Fitting 5 folds for each of 7 candidates, totalling 35 fits


Best AUC: 0.904
Best C: 1


In [81]:
dv, model = train(df_full_train, df_full_train.target.values, C=1)
y_pred = predict(df_test, dv, model)
auc = roc_auc_score(y_test, y_pred)
print(f"AUC on test set: {auc:.3f}\n")

actual_positive = (y_test == 1)
actual_negative = (y_test == 0)

t=0.5
predicted_positive = (y_pred >= t)
predicted_negative = (y_pred < t)

tp = (predicted_positive & actual_positive).sum()
tn = (predicted_negative & actual_negative).sum()
fp = (predicted_positive & actual_negative).sum()
fn = (predicted_negative & actual_positive).sum()

#print(tp)

total_pos = actual_positive.sum()
total_neg = (~actual_positive).sum()

print(f"\nConfusion Matrix (threshold={t}):")
print(f"{'':>15} Predicted Pos  Predicted Neg")
print(f"Actual Pos      {tp} ({tp/total_pos:.2%})    {fn} ({fn/total_pos:.2%})")
print(f"Actual Neg      {fp} ({fp/total_neg:.2%})    {tn} ({tn/total_neg:.2%})")
print(f"\nMetrics:")
print(f"  Sensitivity/Recall: {tp/total_pos:.3f}")
print(f"  Specificity:        {tn/total_neg:.3f}")
print(f"  Precision:          {tp/(tp+fp):.3f}")
print(f"  Accuracy:           {(tp+tn)/(tp+tn+fp+fn):.3f}")

AUC on test set: 0.864


Confusion Matrix (threshold=0.5):
                Predicted Pos  Predicted Neg
Actual Pos      29 (82.86%)    6 (17.14%)
Actual Neg      6 (23.08%)    20 (76.92%)

Metrics:
  Sensitivity/Recall: 0.829
  Specificity:        0.769
  Precision:          0.829
  Accuracy:           0.803


After analyzing the model's performance, we found that the model trained with the original dataset and inverted regularization strength (c=1) showed good performance.

However, when we dropped the duplicated dataset, the model lost some comprehension of the feature dataset. However, this process may be beneficial in other situations.